# Scrapy
- 웹사이트에서 데이터 수집을 위한 오픈소스 파이썬 프레임워크
- 멀티스레딩으로 데이터 수집
- daum 뉴스데이터 수집

In [2]:
# install scrapy
!pip install scrapy --quiet

In [3]:
import scrapy
import requests
from scrapy.http import TextResponse

## 1. make project

In [4]:
!scrapy startproject news

New Scrapy project 'news', using template directory 'C:\Users\User\anaconda3\Lib\site-packages\scrapy\templates\project', created in:
    C:\Users\User\Desktop\kt_aivle\2024.09.23 웹크롤링3\news

You can start your first spider with:
    cd news
    scrapy genspider example example.com


In [7]:
!tree news /F

���� PATH�� ����Դϴ�.
���� �Ϸ� ��ȣ�� 385B-BB36�Դϴ�.
C:\USERS\USER\DESKTOP\KT_AIVLE\2024.09.23 ��ũ�Ѹ�3\NEWS
��  scrapy.cfg
��  
����news
    ��  items.py
    ��  middlewares.py
    ��  pipelines.py
    ��  settings.py
    ��  __init__.py
    ��  
    ����spiders
            __init__.py
            


- scrapy structure
    - items : 데이터의 모양 정의
    - middewares : 수집할때 header 정보와 같은 내용을 설정 (ex. useragent 등)
    - pipelines : 데이터를 수집한 후에 코드를 실행
    - settings : robots.txt 규칙, 크롤링 시간 텀등을 설정 (크롤링 정책을 준수할지 말지)
    - spiders : 크롤링 절차를 정의

## 2. xpath
- link, contents

### xpath
- html element 선택하는 방법
- scrapy 에서는 기본적으로 xpath를 사용
- syntax
    - // : 최상위 엘리먼트
    - \* : 모든 하위 엘리먼트 : css selector의 한칸띄우기와 같다.
    - [@id="value"] : 속성값 선택
    - / : 한단계 하위 엘리먼트 : css selector의 >와 같다.
    - [n] : nth-child(n)

In [10]:
url = 'https://news.daum.net'
response = requests.get(url)
response = TextResponse(response.url, body=response.text, encoding='utf-8') # TextResponse는 BeautifulSoup의 일종이라고 생각할 수 있다.
response, type(response)

(<200 https://news.daum.net/>, scrapy.http.response.text.TextResponse)

In [17]:
selector = "/html/body/div[2]/main/section/div/div[1]/div[1]/ul/li/div/div/strong/a/@href"
links = response.xpath(selector).extract() #extract - 값들만 추출
len(links), links[:5]

(20,
 ['https://v.daum.net/v/20240923151053270',
  'https://v.daum.net/v/20240923150750118',
  'https://v.daum.net/v/20240923145528413',
  'https://v.daum.net/v/20240923144454810',
  'https://v.daum.net/v/20240923135809940'])

In [18]:
link = links[0]
response = requests.get(link)
response = TextResponse(response.url, body=response.text, encoding='utf-8') # 위 각각 기사 링크에 접근
response

<200 https://v.daum.net/v/20240923151053270>

In [21]:
title = response.xpath('//*[@id="mArticle"]/div[1]/h3/text()')[0].extract()
title

"전국 주택 2가구 중 1가구 이상은 '준공 20년 이상'"

## 3. items.py
- Data Model

In [28]:
# %load news/news/items.py

In [27]:
%%writefile news/news/items.py

import scrapy


class NewsItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    title = scrapy.Field()
    link = scrapy.Field()

Overwriting news/news/items.py


## 4. spider.py
- wirte crawling process

In [30]:
%%writefile news/news/spiders/spider.py

import scrapy
from news.items import NewsItem

class NewsSpider(scrapy.Spider):
    name = 'news'
    allow_domain = ['daum.net']
    start_urls = ['https://news.daum.net']
    
    # def start_requests(self):
    #     urls = [
    #         "https://quotes.toscrape.com/page/1/",
    #         "https://quotes.toscrape.com/page/2/",
    #     ]
    #     for url in urls:
    #         yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        # requests는 이미 scrapy.Spider 안에 있음.
        selector = "/html/body/div[2]/main/section/div/div[1]/div[1]/ul/li/div/div/strong/a/@href"
        links = response.xpath(selector).extract() #extract - 값들만 추출

        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = NewsItem()
        item['link'] = response.url
        item['title'] = response.xpath('//*[@id="mArticle"]/div[1]/h3/text()')[0].extract()
        yield item

Overwriting news/news/spiders/spider.py


## 5. run scrapy
- news 디렉토리에서 아래의 커멘드 실행
- scrapy crawl news -o news.csv

In [33]:
# !scrapy crawl news -o news.csv

UsageError: Line magic function `%scrapy` not found.


In [34]:
%pwd

'c:\\Users\\User\\Desktop\\kt_aivle\\2024.09.23 웹크롤링3'